In [7]:
%pip install -qU langchain-qdrant
%pip install langchain_openai
%pip install qdrant-client
%pip install langchain
%pip install langchain-community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached marshmallow-3.23.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.9 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached marshmallow-3.23.2-py3-none-any.whl (49 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
Using cached mypy_extensions-1.0.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain.document_loaders import CSVLoader
from langchain_community.document_loaders import TextLoader
from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

import os
from dotenv import load_dotenv

load_dotenv()


In [9]:
loader = CSVLoader('Mens Fashion.csv')
data = loader.load()
data

[Document(metadata={'source': 'Mens Fashion.csv', 'row': 0}, page_content="name: Sparx mens Sd9039g Sneakers\nmain_category: stores\nsub_category: Men's Fashion\nimage: https://m.media-amazon.com/images/I/51e48JV2GfL._AC_UL320_.jpg\nlink: https://www.amazon.in/Sparx-Casual-Stripped-Sneakers-White/dp/B0B4KF6LFM/ref=sr_1_5?qid=1679150696&s=apparel&sr=1-5\nratings: 4.0\nno_of_ratings: 1,893\ndiscount_price: ₹499\nactual_price: ₹999"),
 Document(metadata={'source': 'Mens Fashion.csv', 'row': 1}, page_content="name: WildHorn Brown RFID Blocking Leather Wallet for Men I Ultra Strong Stitching I 6 Credit Card Slots I 2 Currency Compartmen...\nmain_category: stores\nsub_category: Men's Fashion\nimage: https://m.media-amazon.com/images/I/81vMec1lk+L._AC_UL320_.jpg\nlink: https://www.amazon.in/WildHorn-Brown-Wallet-WH2052-Crackle/dp/B07C3VH84K/ref=sr_1_6?qid=1679150696&s=apparel&sr=1-6\nratings: 4.0\nno_of_ratings: 33,466\ndiscount_price: ₹285\nactual_price: ₹1,499"),
 Document(metadata={'source

In [10]:
vector_embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"),
                                    model="text-embedding-3-large")

qdrant_url = os.getenv("QDRANT_URL")
qdrant_api_key = os.getenv("QDRANT_API_KEY")

Inserting data into Qdrant

In [11]:
qdrant = QdrantVectorStore.from_documents(
    data,
    vector_embeddings,
    url=qdrant_url,
    collection_name="e-commerce AI Recommendation System",
    prefer_grpc=True,
    api_key=qdrant_api_key,
)

Example

In [48]:
products = qdrant.similarity_search("Blue Jeans", k=4)
products

[Document(metadata={'source': 'Mens Fashion.csv', 'row': 5272, '_id': '06cd77b0-b037-444b-b460-3596a6c4e9fd', '_collection_name': 'e-commerce AI Recommendation System'}, page_content="name: Campus Sutra Men’s Classic Blue Dark-Washed Distressed Regular Fit Denim Jeans for Casual Wear | Mid-Rise Stretchable Cott...\nmain_category: stores\nsub_category: Men's Fashion\nimage: https://m.media-amazon.com/images/I/51PAjY776nL._AC_UL320_.jpg\nlink: https://www.amazon.in/Campus-Sutra-Denim-Jeans-AZ18JN_Knees_M_PLN_LBU_AZ_32/dp/B085GBNQQW/ref=sr_1_5281?qid=1679150909&s=apparel&sr=1-5281\nratings: 3.5\nno_of_ratings: 536\ndiscount_price: ₹645\nactual_price: ₹1,699"),
 Document(metadata={'source': 'Mens Fashion.csv', 'row': 5345, '_id': '1149c0fc-f558-47f7-9fb3-757fb0ccdb80', '_collection_name': 'e-commerce AI Recommendation System'}, page_content="name: Levi's Men's 517 Bootcut Jeans\nmain_category: stores\nsub_category: Men's Fashion\nimage: https://m.media-amazon.com/images/I/71OQvqwwxML._AC_U